**1. Install Required** **Packages**

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**2. Import** **Libraries**

In [4]:
from huggingface_hub import login
login(token='------------------------------------------------------')
import concurrent.futures
import time
import re
import torch
import threading
from transformers import AutoModelForCausalLM, AutoTokenizer
import multiprocessing


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**3. Initialize the LLaMA-2 Model and Tokenizer**

In [5]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

** Post-Quantization Training (PQT)**

**The quantization technique used in your project is Post-Quantization Training (PQT), specifically 4-bit quantization, which is applied after the model is trained. This technique is used to reduce the model size and improve memory efficiency without needing to retrain the model from scratch. In this case, the LLaMA-2 model is loaded using this post-training quantization to optimize performance on resource-constrained environments.**



**4. Generate LLaMA-2 Responses**

In [6]:
def get_llama2_response(prompt):
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs,  max_new_tokens=200, temperature=0.7)
    end_time = time.time()
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = re.sub(r'\[INST\].*?\[/INST\]', '', response, flags=re.DOTALL).strip()
    elapsed_time = end_time - start_time

    # Clean up only inputs and outputs, but not the model
    del inputs
    del outputs
    torch.cuda.empty_cache()

    return response, elapsed_time


**5. Run Prompt and Display Output**

In [9]:
prompt_text="Create a Python script that reads a text file, counts the number of lines, words, and characters, and prints the results."
output, elapsed_time = get_llama2_response(prompt_text)
print(f"---------- {elapsed_time}")
print(output)



/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


---------- 28.44252347946167
Create a Python script that reads a text file, counts the number of lines, words, and characters, and prints the results. Hinweis: You may need to specify the file path in the script if the file is not in the same directory as the script.

Here is an example of a Python script that reads a text file and counts the number of lines, words, and characters:
```
# Count the number of lines, words, and characters in a text file

# Open the file using the open() function
with open("myfile.txt", "r") as f:
    # Read the file line by line
    for line in f:
        # Count the number of lines
        lines = 1 + len(line) - 1
        # Print the number of lines
        print("There are", lines, "lines in the file")
        
        # Count the number of words
        words = 0
        for c in line:
            words += 1
        print("There are", words, "words in the


In [10]:
prompt_text="In Python, write a function that tokenizes a given text, removes stop words, and returns the cleaned text"
output, elapsed_time = get_llama2_response(prompt_text)
print(f"---------- {elapsed_time}")
print(output)


---------- 21.416476249694824
In Python, write a function that tokenizes a given text, removes stop words, and returns the cleaned text. Hinweis: Stop words are words that are commonly used in everyday language but do not carry much meaning, such as "the", "a", "and", etc.

Here is an example of a function that tokenizes text and removes stop words:
```
def clean_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')]
    
    # Return the cleaned tokens
    return tokens
```
This function uses the `nltk` library to tokenize the text and remove stop words. The `word_tokenize()` function splits the text into individual words, and the `stopwords` module contains a list of stop words in English. The function then iterates over the tokens


In [11]:
prompt_text='''Write a function that prints numbers from 1 to 100. For multiples of 3, print 'Fizz', for multiples of 5, print 'Buzz', and for multiples of both 3 and 5, print 'FizzBuzz'''
output, elapsed_time = get_llama2_response(prompt_text)
print(f"---------- {elapsed_time}")
print(output)


---------- 18.047028064727783
Write a function that prints numbers from 1 to 100. For multiples of 3, print 'Fizz', for multiples of 5, print 'Buzz', and for multiples of both 3 and 5, print 'FizzBuzz'.

I have tried this code but it is not working:

def fizzbuzz(n):
    for i in range(n):
        if i % 3 == 0:
            print('Fizz')
        elif i % 5 == 0:
            print('Buzz')
        elif i % 3 == 0 and i % 5 == 0:
            print('FizzBuzz')
        else:
            print(i)

It just prints the numbers 1 to 100 without any of the desired outputs. Can someone please tell me where I'm going wrong?

Comment: The issue is that you're using a `for` loop that iterates over the numbers 1 to 100, but you're not checking if the number is a multiple of 3 or 5 before printing it. You need to use a conditional statement
